# Automatic Recognition of Vehicles at Intersections and Update of Moving Vehicles
 


### Monitoring Execution
Using the time library we will monitor our programms execution.

In [1]:
#Start counting the time execution of our programm.
%run timer_class.ipynb
timer = Timer()
timer.start()



Firstly, we will create a function that will check the existense of all necessary libraries.So as to ensure a stable execution of our programm.

In [2]:
import sys 
import os

%run check_env.ipynb
check_env()

In [3]:
#Now we are sure for the existence of the libraries. We can safely proceed with importing them.
import numpy as np
import cv2 
import tensorflow as tf
import matplotlib as plt
import pandas as pd


import warnings 
warnings.filterwarnings("ignore")


In [4]:
#do an environment check
df = pd.Dataframe(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),columns=['a', 'b', 'c'])

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [ ]:
#Stop counting time see how much time it took for the libraries to get imported.
timer.stop()